### Editering av enkel dataframe 

#### Imports

In [ ]:
from ssb_dash_framework import main_layout
from ssb_dash_framework import app_setup
from ssb_dash_framework import set_variables
from ssb_dash_framework import EditingTableTab # Tab modulen
from ssb_dash_framework import EditingTableWindow # Vindu modulen
from ssb_dash_framework import apply_edits

import pandas as pd
import numpy as np

import os
import uuid
from datetime import datetime, timezone

from fagfunksjoner import next_version_path

#### Les inn data

In [ ]:
filsti_data = '/buckets/produkt/editering/editering_test_v1.parquet'

In [ ]:
df = pd.read_parquet(
    filsti_data
)

In [ ]:
filsti_logg = '/buckets/produkt/editering/logg/prosessdata/prosess-editering_test_v1.jsonl'
try: os.makedirs('/buckets/produkt/editering/logg/prosessdata') # lag mappe for logg 
except: pass

#### Klargjøre data

Editeringen avhenger av at man har en *uuid*-kolonne i dataframen. Dette er for å forsikre at riktig rad blir endret.

In [ ]:
if 'uuid' not in df.columns: df['uuid'] = [uuid.uuid4() for _ in range(len(df))]

### Kjør app

#### App setup 

In [ ]:
port = 8070
service_prefix = os.getenv("JUPYTERHUB_SERVICE_PREFIX", "/")
domain = os.getenv("JUPYTERHUB_HTTP_REFERER", None)
theme = "cosmo" # sjekk ut flere temaer: https://www.dash-bootstrap-components.com/docs/themes/explorer/ 
app = app_setup(port, service_prefix, theme)
app_timestamp = datetime.now()

#### Velg variabler 

In [ ]:
set_variables(["orgnr","aar"]) # Dette gjør at orgnr og aar er tilgjengelig i applikasjonen din.
start_verdier = { # Valgfritt å ha med, men kan være praktisk for brukervennlighet. Puttes inn i main_layout() funksjonen.
    "orgnr": "971526920", 
    "aar": "2020"
}

# Definer get_data funkjson
def get_data(orgnr,aar):
    return df[(df['orgnr'] == orgnr) & (df['aar']==aar)]
    
enhetstabell = EditingTableTab(
    label="Enhetstabell",
    inputs=["orgnr", "aar"], # evt. start_verdier.keys()
    states=[],
    get_data_func=get_data,
    output="aar",
    log_filepath = filsti_logg,
    justify_edit=False
    
)

tab_list = [
    enhetstabell,
]
window_list = []

app.layout = main_layout(window_list, tab_list, default_values = start_verdier, )

if __name__ == "__main__":
    app.run(
        port=port,
        jupyter_server_url=domain,
        jupyter_mode="tab",
    )

#### 

#### Påføre endringer fra logg

In [ ]:
logg = pd.read_json(filsti_logg, lines = True)

print(logg)

In [ ]:
df_editert = apply_edits(df, filsti_logg, app_timestamp)

#### Sjekk hva som har blitt endret 

In [ ]:
pd.concat([df,df_editert]).drop_duplicates(keep=False)

#### Lagre det editerte datasettet

In [ ]:
filsti_data_neste_versjon = next_version_path(filsti_data)
print(filsti_data_neste_versjon)

In [ ]:
df.to_parquet(filsti_data_neste_versjon)